<h1>Anotador de Podócitos (Versão Python)</h1>

In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 10 02:24:18 2020

@author: george
"""
#Garantindo a importação e instalação do pacote tkinter 
try:
    import tkinter as tk
    from tkinter import ttk, filedialog, Label, Button, Menu
    from tkinter.messagebox import showinfo
except ImportError:
    from pip._internal import main as pip
    pip(['install', '--user', 'python-tk'])
    import tkinter as tk
    from tkinter import ttk, filedialog, Label, Button, Menu
    from tkinter.messagebox import showinfo

#Garantindo a importação e instalação do pacote PIL
try:
    from PIL import ImageTk, Image
    from PIL import _tkinter_finder
except ImportError:
    from pip._internal import main as pip
    pip(['install', '--user', 'pil'])
    from PIL import ImageTk, Image
    from PIL import _tkinter_finder ##Sem essa importação dá erro no executável


#import tkFileDialog

filename = 0
array_anotacao=[]#armazenar aqui coordenadas iniciais e finais de cada retângulo
only_name=""
array_save_rets = []
class App(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self._createVariables(parent)
        self._createCanvas()
        self._createCanvasBinding()
            
    def _createVariables(self, parent):
        self.parent = parent
        self.rectx0 = 0
        self.recty0 = 0
        self.rectx1 = 0
        self.recty1 = 0
        self.rectid = None
        self.move = False
        
    def _createCanvas(self):
        canvas_w= 800
        canvas_h = 600
        global filename
        self.canvas = tk.Canvas(self.parent, width = canvas_w, height = canvas_h)
        self.canvas.pack(expand = "yes", fill = "both")#self.canvas.grid(row=0, column=0, sticky="nsew")
        self.im = ImageTk.PhotoImage(file = filename)
        self.im.width = canvas_w
        self.im.height = canvas_h
        self.canvas.create_image(0, 0, image = self.im, anchor = "nw")
        #self.canvas.create_image(10, 10, image = im, anchor = 'nw')
        self.canvas.focus_set()
  
    def _createCanvasBinding(self):
        self.canvas.bind( "<Button-1>", self.startRect )
        self.canvas.bind( "<ButtonRelease-1>", self.stopRect )
        self.canvas.bind( "<Motion>", self.movingRect )
        self.canvas.bind("<KeyPress>",self.atalhos)
    
    def atalhos(self, event):
        #print event.char
        if (event.char=="s"): 
            geraTxt()
        if (event.char=="d"): 
            self.deleteRet()
        if (event.char=="h"): 
            popup_ajuda()
        if (event.char==""): 
            array_anotacao.clear()
            self.parent.destroy()
            
    
    #Método para excluir os retângulos
    def deleteRet(self):
        #showinfo("Window", "Testando")
        #deletando os rets pelo ordm de que foram armazenados no array 
        global array_save_rets
        self.size = len(array_save_rets)
        self.canvas.delete(array_save_rets[self.size-1]) #excluindo o retangulo
        
        #eliminando o registro do retangulo no array
        del(array_save_rets[self.size-1])
        #self.canvas.delete(self.rect)
        
        #preciso também excluir as coordenadas do retangulo que foram armaznadas no array de coordenadas, que será registrado no txt
        global array_anotacao
        self.size_array_anotacao = len(array_anotacao)
        del(array_anotacao[self.size_array_anotacao-1])
        del(array_anotacao[self.size_array_anotacao-2])
        print ("Retangulo excluído")
    
    def startRect(self, event):
        self.move = True
        
        #Translate mouse screen x0,y0 coordinates to canvas coordinates
        self.rectx0 = self.canvas.canvasx(event.x)
        self.recty0 = self.canvas.canvasy(event.y)
        
        #Create rectangle
        self.rect = self.canvas.create_rectangle(
        self.rectx0, self.recty0, self.rectx0, self.recty0, outline='yellow', width=2) #fill="#ffffff"
        
        #save os rects em um array para depois excluí-los na ordem de criação
        global array_save_rets
        array_save_rets = array_save_rets + [self.rect]
        
        #Get rectangle"s canvas object ID
        self.rectid = self.canvas.find_closest(self.rectx0, self.recty0, halo=2)
        print("Rectangle {0} started at {1} {2} {3} {4} ".
        format(self.rect, self.rectx0, self.recty0, self.rectx0,
        self.recty0))
        #global array_temp
        #print self.rect
        global array_anotacao
        array_anotacao=array_anotacao+[[self.rectx0,self.recty0]]
    
    def movingRect(self, event):
        if self.move:
            #Translate mouse screen x1,y1 coordinates to canvas coordinates
            self.rectx1 = self.canvas.canvasx(event.x)
            self.recty1 = self.canvas.canvasy(event.y)
            #Modify rectangle x1, y1 coordinates
            self.canvas.coords(self.rectid, self.rectx0, self.recty0,
            self.rectx1, self.recty1)
            #print("Rectangle x1, y1 = ", self.rectx1, self.recty1)
        
    def stopRect(self, event):
        self.move = False
        #Translate mouse screen x1,y1 coordinates to canvas coordinates
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        #Modify rectangle x1, y1 coordinates (final)
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
        self.rectx1, self.recty1)
        global array_anotacao
        array_anotacao=array_anotacao+[[self.rectx1,self.recty1]]
        print("Rectangle x1, y1 = ", self.rectx1, self.recty1)
        print("Rectangle ended")
        
def UploadAction():
    global filename
    filename= filedialog.askopenfilename()
    print('Selected:', filename)
    
    #redimensionando a imagem e salvando em uma nova dimensão
    #basewidth = 300
    img = Image.open(filename)
    #wpercent = (basewidth/float(img.size[0]))
    #hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((800,600), Image.ANTIALIAS)
    img.save(filename) 

    nome_da_foto = open(filename) #peguei aqui noamente pra conseguir depois acessar apenas o nome e gerar um txt com mesmo título
    # Explode o arquivo em cada ponto (.)
    explod_file = nome_da_foto.name.split('.')
    # Apenas o nome do arquivo
    global only_name
    only_name = '.'.join(explod_file[:-1])
    print(only_name)

    root2 = tk.Toplevel()
    #root = tk.Tk()
    menubar = Menu(root2, background='white', foreground='black', activebackground='#004c99', activeforeground='white')
    root2.config(menu=menubar)
    root2.geometry( "800x600" )
    root2.title("Anotador de Podócitos")
    app = App(root2)
    
    menubar.add_command(label='Desfazer (d)', command=app.deleteRet)
    menubar.add_separator()
    menubar.add_command(label='Salvar (s)', command=geraTxt)
    menubar.add_separator()
    menubar.add_command(label='Ajuda (h)', command=popup_ajuda)
    menubar.add_separator()
    menubar.add_command(label='Sair (Esc)', command=root2.destroy)      
    root2.mainloop()

def popup_ajuda():
    win = tk.Toplevel()
    win.wm_title("Instruções de Utilização")
    
    win.geometry("600x280")
    l1 = tk.Label(win, wraplength=580, text="(1) Clique uma vez nas mediações do podócito para criar o ponto inicial, clique uma segundo vez em um segundo ponto. Um retângulo será formado. (2) Após demarcar as regiões de todos os podócitos, clique em salvar. (3) Neste momento a marcação já foi salva, agora você pode fechar a tela ou cicar na opção sair. (4) Escolha outra imagem e repita o processo.", font=('Verdana','12','bold'))
    l1.grid(row=0, column=0)
    
    l2 = tk.Label(win, pady=15, wraplength=580, text="Caso você tenha marcado um podócito errado, poderá desfazer a marcação clicando no botão _Desfazer_", font=('Arial','12', 'bold'))
    l2.grid(row=1, column=0)
     
    b = ttk.Button(win, text="Okay", command=win.destroy)
    b.grid(row=5, column=0)

def geraTxt():
    global array_anotacao
    array_anotacao
    array_temp = str(array_anotacao)
    global only_name
    only_name = only_name+".txt"
    arquivo = open(only_name, 'w')
    arquivo.write(array_temp)
    arquivo.close()
    array_anotacao = []
    showinfo("Obrigado!", "Anotação salva com sucesso!")
    
class Application(ttk.Frame):

    def __init__(self, master):
        ttk.Frame.__init__(self, master)
        self.pack()
        
        self.titulo=Label(self, text = 'Bem vindo! Por favor, clique no botão para escolher uma imagem ', font=('Verdana','12','bold'), padx=5, pady=15)
        self.titulo.pack()        
        
        self.b01=Button(self,text='Escolher imagem', font=('Verdana','12',), command=UploadAction)
        self.b01['padx'],self.b01['pady'] = 10, 5
        self.b01['bg']='gray'
        self.b01.pack()

root = tk.Tk()
root.title("Pathospotter - Anotação de Imagens")
root.geometry("800x100")
app = Application(root)

root.mainloop()

('Selected:', '/home/george/anotacao natanael/original/66.jpg')
/home/george/anotacao natanael/original/66
Rectangle 2 started at 547.0 61.0 547.0 61.0 
('Rectangle x1, y1 = ', 527.0, 81.0)
Rectangle ended
Rectangle 3 started at 515.0 95.0 515.0 95.0 
('Rectangle x1, y1 = ', 537.0, 76.0)
Rectangle ended
Rectangle 4 started at 262.0 376.0 262.0 376.0 
('Rectangle x1, y1 = ', 294.0, 408.0)
Rectangle ended
Rectangle 5 started at 227.0 418.0 227.0 418.0 
('Rectangle x1, y1 = ', 252.0, 443.0)
Rectangle ended
Rectangle 6 started at 247.0 452.0 247.0 452.0 
('Rectangle x1, y1 = ', 271.0, 418.0)
Rectangle ended
Rectangle 7 started at 443.0 534.0 443.0 534.0 
('Rectangle x1, y1 = ', 400.0, 570.0)
Rectangle ended
Rectangle 8 started at 530.0 180.0 530.0 180.0 
('Rectangle x1, y1 = ', 501.0, 210.0)
Rectangle ended
Rectangle 9 started at 700.0 381.0 700.0 381.0 
('Rectangle x1, y1 = ', 667.0, 413.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/67.jpg')
/home/george/anotac

('Rectangle x1, y1 = ', 374.0, 278.0)
Rectangle ended
Rectangle 10 started at 336.0 253.0 336.0 253.0 
('Rectangle x1, y1 = ', 358.0, 278.0)
Rectangle ended
Rectangle 11 started at 341.0 270.0 341.0 270.0 
('Rectangle x1, y1 = ', 363.0, 298.0)
Rectangle ended
Rectangle 12 started at 230.0 314.0 230.0 314.0 
('Rectangle x1, y1 = ', 250.0, 336.0)
Rectangle ended
Rectangle 13 started at 273.0 395.0 273.0 395.0 
('Rectangle x1, y1 = ', 294.0, 374.0)
Rectangle ended
Rectangle 14 started at 299.0 384.0 299.0 384.0 
('Rectangle x1, y1 = ', 320.0, 406.0)
Rectangle ended
Rectangle 15 started at 312.0 401.0 312.0 401.0 
('Rectangle x1, y1 = ', 331.0, 421.0)
Rectangle ended
Rectangle 16 started at 333.0 433.0 333.0 433.0 
('Rectangle x1, y1 = ', 352.0, 413.0)
Rectangle ended
Rectangle 17 started at 557.0 274.0 557.0 274.0 
('Rectangle x1, y1 = ', 538.0, 306.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/73.jpg')
/home/george/anotacao natanael/original/73
Rectangle 2 sta

/home/george/anotacao natanael/original/80
Rectangle 2 started at 163.0 102.0 163.0 102.0 
('Rectangle x1, y1 = ', 145.0, 132.0)
Rectangle ended
Rectangle 3 started at 104.0 164.0 104.0 164.0 
('Rectangle x1, y1 = ', 134.0, 187.0)
Rectangle ended
Rectangle 4 started at 377.0 147.0 377.0 147.0 
('Rectangle x1, y1 = ', 401.0, 169.0)
Rectangle ended
Rectangle 5 started at 380.0 265.0 380.0 265.0 
('Rectangle x1, y1 = ', 404.0, 286.0)
Rectangle ended
Rectangle 6 started at 335.0 289.0 335.0 289.0 
('Rectangle x1, y1 = ', 310.0, 321.0)
Rectangle ended
Rectangle 7 started at 287.0 322.0 287.0 322.0 
('Rectangle x1, y1 = ', 311.0, 348.0)
Rectangle ended
Rectangle 8 started at 289.0 317.0 289.0 317.0 
('Rectangle x1, y1 = ', 261.0, 347.0)
Rectangle ended
Rectangle 9 started at 130.0 314.0 130.0 314.0 
('Rectangle x1, y1 = ', 105.0, 340.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/81-a.jpg')
/home/george/anotacao natanael/original/81-a
('Selected:', '/home/george/an

('Rectangle x1, y1 = ', 404.0, 308.0)
Rectangle ended
Rectangle 7 started at 398.0 299.0 398.0 299.0 
('Rectangle x1, y1 = ', 420.0, 330.0)
Rectangle ended
Retangulo excluído
Rectangle 8 started at 401.0 331.0 401.0 331.0 
('Rectangle x1, y1 = ', 420.0, 303.0)
Rectangle ended
Retangulo excluído
Rectangle 9 started at 421.0 307.0 421.0 307.0 
('Rectangle x1, y1 = ', 395.0, 334.0)
Rectangle ended
Rectangle 10 started at 362.0 394.0 362.0 394.0 
('Rectangle x1, y1 = ', 387.0, 420.0)
Rectangle ended
Rectangle 11 started at 159.0 470.0 159.0 470.0 
('Rectangle x1, y1 = ', 186.0, 495.0)
Rectangle ended
Rectangle 12 started at 66.0 349.0 66.0 349.0 
('Rectangle x1, y1 = ', 89.0, 371.0)
Rectangle ended
Rectangle 13 started at 94.0 290.0 94.0 290.0 
('Rectangle x1, y1 = ', 125.0, 317.0)
Rectangle ended
Rectangle 14 started at 152.0 233.0 152.0 233.0 
('Rectangle x1, y1 = ', 178.0, 260.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/85.jpg')
/home/george/anotacao natana

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/george/anaconda2/lib/python2.7/lib-tk/Tkinter.py", line 1547, in __call__
    return self.func(*args)
  File "<ipython-input-1-ea52fcc72726>", line 159, in UploadAction
    img = Image.open(filename)
  File "/home/george/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 2658, in open
    fp = io.BytesIO(fp.read())
AttributeError: 'unicode' object has no attribute 'read'


('Selected:', '/home/george/anotacao natanael/original/91.jpg')
/home/george/anotacao natanael/original/91
Rectangle 2 started at 341.0 45.0 341.0 45.0 
('Rectangle x1, y1 = ', 366.0, 67.0)
Rectangle ended
Rectangle 3 started at 366.0 45.0 366.0 45.0 
('Rectangle x1, y1 = ', 396.0, 70.0)
Rectangle ended
Rectangle 4 started at 248.0 70.0 248.0 70.0 
('Rectangle x1, y1 = ', 227.0, 95.0)
Rectangle ended
Rectangle 5 started at 103.0 218.0 103.0 218.0 
('Rectangle x1, y1 = ', 97.0, 247.0)
Rectangle ended
Retangulo excluído
Rectangle 6 started at 86.0 220.0 86.0 220.0 
('Rectangle x1, y1 = ', 109.0, 252.0)
Rectangle ended
Rectangle 7 started at 73.0 277.0 73.0 277.0 
('Rectangle x1, y1 = ', 111.0, 315.0)
Rectangle ended
Rectangle 8 started at 432.0 190.0 432.0 190.0 
('Rectangle x1, y1 = ', 460.0, 217.0)
Rectangle ended
Rectangle 9 started at 454.0 211.0 454.0 211.0 
('Rectangle x1, y1 = ', 475.0, 236.0)
Rectangle ended
Rectangle 10 started at 313.0 432.0 313.0 432.0 
('Rectangle x1, y1 = ',

Retangulo excluído
Rectangle 6 started at 207.0 314.0 207.0 314.0 
('Rectangle x1, y1 = ', 245.0, 349.0)
Rectangle ended
Rectangle 7 started at 370.0 561.0 370.0 561.0 
('Rectangle x1, y1 = ', 395.0, 586.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/98.jpg')
/home/george/anotacao natanael/original/98
Rectangle 2 started at 141.0 168.0 141.0 168.0 
('Rectangle x1, y1 = ', 162.0, 192.0)
Rectangle ended
Rectangle 3 started at 138.0 192.0 138.0 192.0 
('Rectangle x1, y1 = ', 160.0, 214.0)
Rectangle ended
Rectangle 4 started at 671.0 317.0 671.0 317.0 
('Rectangle x1, y1 = ', 643.0, 340.0)
Rectangle ended
Rectangle 5 started at 629.0 404.0 629.0 404.0 
('Rectangle x1, y1 = ', 655.0, 432.0)
Rectangle ended
Rectangle 6 started at 450.0 525.0 450.0 525.0 
('Rectangle x1, y1 = ', 413.0, 555.0)
Rectangle ended
Rectangle 7 started at 366.0 545.0 366.0 545.0 
('Rectangle x1, y1 = ', 326.0, 568.0)
Rectangle ended
Rectangle 8 started at 259.0 494.0 259.0 494.0 
('Rectangl